In [1]:
import pandas as pd
import requests
import random
#中商产业研究院网址
url = 'http://s.askci.com/stock/a/?reportTime=2018-09-30&pageNum=%d'
#伪装浏览器
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36',
    'Referer': 'http://s.askci.com/stock/a/?reportTime=2018-09-30&pageNum=178'
}
IPs = [{'HTTP': 'HTTP://118.190.95.35:9001'},
       {'HTTP': 'HTTP://61.135.217.7:80'},
       {'HTTP': 'HTTP://116.1.11.19:80'}]
for i in range(1, 179):
    # 返回整个网页中的表格列表
    dflist = pd.read_html(requests.get(url % i, 
            headers=headers, proxies=random.choice(IPs)).text)
    # 经观察可以发现是第四个就是我们想要的表格
    df = dflist[3]
    df.to_csv('stock_information.csv', mode='a', 
              header=1 if i == 1 else None, 
              index=0, encoding='utf_8_sig')

In [2]:
#先引入后面可能用到的包（package）
import pandas as pd  
import numpy as np
from scipy import stats
import tushare as ts 
import matplotlib.pyplot as plt
%matplotlib inline   

#正常显示画图时出现的中文和负号
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

In [3]:
#导入数据
df=pd.read_csv('stock_information.csv')

In [5]:
#查看数据基本情况
df.head()
#利润和主营业务收入百万元

,序号,股票代码,股票简称,公司名称,省份,城市,主营业务收入(201809),净利润(201809),员工人数,上市日期,招股书,公司财报,行业分类,产品类型,主营业务
0,1,1,平安银行,平安银行股份有限公司,深圳市,罗湖区,8666400.0,2045600.0,32744,1991-04-03,--,NaN,银行,商业银行业务,经有关监管机构批准的各项商业银行业务
1,2,2,万科A,万科企业股份有限公司,深圳市,盐田区,17602213.92,1398455.98,92764,1991-01-29,--,NaN,房地产开发,房地产、物业管理、投资咨询,房地产开发和物业服务
2,3,4,国农科技,深圳中国农大科技股份有限公司,深圳市,南山区,25617.86,-259.18,172,1991-01-14,--,NaN,生物医药,生物医药业务,生物制药
3,4,5,世纪星源,深圳世纪星源股份有限公司,深圳市,罗湖区,38300.0,653.2,705,1990-12-10,--,NaN,环保工程、物业管理,酒店经营、物业管理收入、环保业务收入,绿色低碳城市社区建设相关的服务业务
4,5,6,深振业A,深圳市振业(集团)股份有限公司,深圳市,罗湖区,159389.24,50324.01,370,1992-04-27,--,NaN,房地产开发,物业租赁、房产销售,从事房地产开发与销售


In [6]:
#保留需要的特征
df=df.iloc[:,[2,4,6,7,8,9,12,13]]
df.head()

,股票简称,省份,主营业务收入(201809),净利润(201809),员工人数,上市日期,行业分类,产品类型
0,平安银行,深圳市,8666400.0,2045600.0,32744,1991-04-03,银行,商业银行业务
1,万科A,深圳市,17602213.92,1398455.98,92764,1991-01-29,房地产开发,房地产、物业管理、投资咨询
2,国农科技,深圳市,25617.86,-259.18,172,1991-01-14,生物医药,生物医药业务
3,世纪星源,深圳市,38300.0,653.2,705,1990-12-10,环保工程、物业管理,酒店经营、物业管理收入、环保业务收入
4,深振业A,深圳市,159389.24,50324.01,370,1992-04-27,房地产开发,物业租赁、房产销售


In [7]:
#列重命名
df.rename(columns={'省份':'城市','主营业务收入(201809)':'主营业务收入', '净利润(201809)':'净利润'}, inplace = True)
df.head()

,股票简称,城市,主营业务收入,净利润,员工人数,上市日期,行业分类,产品类型
0,平安银行,深圳市,8666400.0,2045600.0,32744,1991-04-03,银行,商业银行业务
1,万科A,深圳市,17602213.92,1398455.98,92764,1991-01-29,房地产开发,房地产、物业管理、投资咨询
2,国农科技,深圳市,25617.86,-259.18,172,1991-01-14,生物医药,生物医药业务
3,世纪星源,深圳市,38300.0,653.2,705,1990-12-10,环保工程、物业管理,酒店经营、物业管理收入、环保业务收入
4,深振业A,深圳市,159389.24,50324.01,370,1992-04-27,房地产开发,物业租赁、房产销售


In [19]:
df.sort_values('净利润',ascending=False)[:20]

,股票简称,城市,主营业务收入,净利润,员工人数,上市日期,行业分类,产品类型
1956,赛托生物,菏泽市,84714.02,9976.96,622,2017-01-06,化学原料药,甾体药物
1227,普路通,深圳市,454925.21,9976.3,447,2015-06-29,供应链管理,供应链管理服务
2421,红星发展,安顺市,114946.68,9975.43,2854,2001-03-20,无机盐,钡盐、锶盐、锰系产品
2835,航天长峰,北京市,90120.49,996.96,1074,1994-04-25,专用计算机设备,安保科技、医疗器械、电子信息
1115,利君股份,成都市,38252.72,9951.93,767,2012-01-06,矿山冶金机械,制造业
2502,中国动力,保定市,2014545.95,99464.51,19716,2004-07-14,船舶制造,制造业
2402,航天动力,西安市,88796.9,994.96,2483,2003-04-08,航天装备,泵及泵系统、液力变矩器、智能燃气表和智能数据模块、电机、建筑安装、化工生物装备、节能项目
2845,百大集团,杭州市,52450.9,9935.14,290,1994-08-09,百货,商品销售、旅游服务
541,沙钢股份,苏州市,1101955.32,99345.68,4339,2006-10-25,普钢,黑色金属冶炼、优特钢
2886,湖南盐业,湖南,165223.94,9933.93,4472,2018-03-26,--,制盐、食用盐、工业盐、芒硝
